<a href="https://colab.research.google.com/github/senasung37/movie_recommender/blob/main/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터 불러오기 & EDA

In [6]:
#라이브러리
import csv
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
#데이터 불러오기
movies = pd.read_csv("movies_metadata.csv", encoding='latin1')
movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,d,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,Toy Story,FALSE,7.7,5415,NaN,NaN,NaN,NaN,NaN
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Roll the dice and unleash the excitement!,Jumanji,FALSE,6.9,2413,NaN,NaN,NaN,NaN,NaN
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,FALSE,6.5,92,NaN,NaN,NaN,NaN,NaN


In [38]:
#데이터프레임 자르기
df_movies = movies.iloc[:, :24]
df_movies.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,FALSE,7.7,5415
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,FALSE,6.9,2413
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,FALSE,6.5,92
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,FALSE,6.1,34
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,FALSE,5.7,173
5,FALSE,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818,170,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,FALSE,7.7,1886
6,FALSE,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0,127,"[{'iso_639_1': 'fr', 'name': 'FranÃ§ais'}, {'i...",Released,You are cordially invited to the most surprisi...,Sabrina,FALSE,6.2,141
7,FALSE,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0,97,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,FALSE,5.4,45
8,FALSE,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171,106,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,FALSE,5.5,174
9,FALSE,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034,130,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,FALSE,6.6,1194


In [11]:
#데이터 불러오기
df_ratings = pd.read_csv("ratings_small.csv", encoding='latin1')
df_ratings.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [12]:
#데이터 인포 확인
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44746 entries, 0 to 44745
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   adult                  44746 non-null  object
 1   belongs_to_collection  4387 non-null   object
 2   budget                 44746 non-null  object
 3   genres                 44746 non-null  object
 4   homepage               7630 non-null   object
 5   id                     44746 non-null  object
 6   imdb_id                44730 non-null  object
 7   original_language      44735 non-null  object
 8   original_title         44746 non-null  object
 9   overview               43793 non-null  object
 10  popularity             44740 non-null  object
 11  poster_path            44360 non-null  object
 12  production_companies   44734 non-null  object
 13  production_countries   44741 non-null  object
 14  release_date           44656 non-null  object
 15  revenue            

In [13]:
#데이터 인포 확인
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [14]:
#null 개수 확인
df_movies.isna().sum()

adult                        0
belongs_to_collection    40359
budget                       0
genres                       0
homepage                 37116
id                           0
imdb_id                     16
original_language           11
original_title               0
overview                   953
popularity                   6
poster_path                386
production_companies        12
production_countries         5
release_date                90
revenue                      8
runtime                    286
spoken_languages            72
status                     235
tagline                  24287
title                       24
video                       48
vote_average               115
vote_count                 309
dtype: int64

In [39]:
#분석에 필요한 열 선택
cols = ['id', 'title', 'genres', 'original_language', 'release_date', 'runtime', 'tagline']
df = df_movies[cols]

In [25]:
df.head(3)

,id,title,genres,original_language,release_date,runtime,tagline
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,1995-10-30,81,NaN
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,1995-12-15,104,Roll the dice and unleash the excitement!
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,1995-12-22,101,Still Yelling. Still Fighting. Still Ready for...


In [43]:
df['genres_2'] = df['genres'].str[20:28]
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,title,genres,original_language,release_date,runtime,tagline,genres_2
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,1995-10-30,81,NaN,'Animati
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,1995-12-15,104,Roll the dice and unleash the excitement!,'Adventu
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,1995-12-22,101,Still Yelling. Still Fighting. Still Ready for...,': 'Roma
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,1995-12-22,127,Friends are the people who let you be yourself...,'Comedy'
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",en,1995-02-10,106,Just When His World Is Back To Normal... He's ...,'Comedy'
...,...,...,...,...,...,...,...,...
44741,439050,Subdue,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",fa,NaN,90,Rising and falling between a man and woman,'Drama'}
44742,111109,Century of Birthing,"[{'id': 18, 'name': 'Drama'}]",tl,2011-11-17,360,NaN,'Drama'}
44743,67758,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",en,2003-08-01,90,A deadly game of wits.,'Action'
44744,227506,Satan Triumphant,[],en,1917-10-21,87,NaN,
